In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcard-csv/creditcard.csv
/kaggle/input/creditcardfraud/creditcard.csv
/kaggle/input/credit-card-project/creditcard.csv


In [69]:
import pandas as pd

# Load the dataset (ensure the file path is correct after uploading)
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

# Display the first few rows of the dataset
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [70]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

# Check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Normalize the 'Amount' feature
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])

# Normalize the 'Time' feature
df['Time'] = scaler.fit_transform(df[['Time']])

# Display the first few rows of the dataset after normalization
df.head()

# Handle class imbalance using undersampling
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0]

# Perform undersampling
non_fraud_sample = non_fraud.sample(n=len(fraud))
balanced_df = pd.concat([fraud, non_fraud_sample], axis=0)

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

# Display the distribution of the 'Class' feature
print("Distribution of 'Class' feature in balanced dataset:\n", balanced_df['Class'].value_counts())


Missing values in each column:
 Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
Distribution of 'Class' feature in balanced dataset:
 Class
1    492
0    492
Name: count, dtype: int64


In [71]:
from sklearn.model_selection import train_test_split

# Define features and target
X = balanced_df.drop('Class', axis=1)
y = balanced_df['Class']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


Training set shape: (787, 30)
Testing set shape: (197, 30)


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_recall_curve, auc

# Initialize the logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Evaluate the model's performance
print("Classification Report:\n", classification_report(y_test, y_pred))

# Calculate Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc)


Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.93        93
           1       0.95      0.92      0.94       104

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9780179685673107


In [73]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the model's performance
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

# Calculate Precision-Recall AUC
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_prob_rf)
pr_auc_rf = auc(recall_rf, precision_rf)
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc_rf)


Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93        93
           1       0.98      0.89      0.93       104

    accuracy                           0.93       197
   macro avg       0.94      0.94      0.93       197
weighted avg       0.94      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9775463786319708


In [74]:
from sklearn.metrics import roc_auc_score

# Evaluate Logistic Regression
print("Logistic Regression:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc)
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

# Evaluate Random Forest
print("\nRandom Forest:")
print("Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc_rf)
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_rf))


Logistic Regression:
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.93        93
           1       0.95      0.92      0.94       104

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9780179685673107
ROC AUC Score: 0.9646401985111662

Random Forest:
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93        93
           1       0.98      0.89      0.93       104

    accuracy                           0.93       197
   macro avg       0.94      0.94      0.93       197
weighted avg       0.94      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9775463786319708
ROC AUC Score: 0.964485111662531


In [75]:
# Apply undersampling
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

# Train and evaluate Logistic Regression with undersampling
model_under = LogisticRegression(max_iter=1000, random_state=42)
model_under.fit(X_train_under, y_train_under)
y_pred_under = model_under.predict(X_test)
y_prob_under = model_under.predict_proba(X_test)[:, 1]

print("Logistic Regression with Undersampling:")
print("Classification Report:\n", classification_report(y_test, y_pred_under))
precision_under, recall_under, _ = precision_recall_curve(y_test, y_prob_under)
pr_auc_under = auc(recall_under, precision_under)
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc_under)
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_under))

# Train and evaluate Random Forest with undersampling
rf_model_under = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_under.fit(X_train_under, y_train_under)
y_pred_rf_under = rf_model_under.predict(X_test)
y_prob_rf_under = rf_model_under.predict_proba(X_test)[:, 1]

print("\nRandom Forest with Undersampling:")
print("Classification Report:\n", classification_report(y_test, y_pred_rf_under))
precision_rf_under, recall_rf_under, _ = precision_recall_curve(y_test, y_prob_rf_under)
pr_auc_rf_under = auc(recall_rf_under, precision_rf_under)
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc_rf_under)
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_rf_under))


Logistic Regression with Undersampling:
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.93        93
           1       0.95      0.92      0.94       104

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9780512477420173
ROC AUC Score: 0.9646401985111662

Random Forest with Undersampling:
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.97      0.93        93
           1       0.97      0.89      0.93       104

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9781914537051439
ROC AUC Score: 0.9668631100082713


In [76]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize Grid Search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='roc_auc', n_jobs=-1)

# Fit Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred_best = best_model.predict(X_test)
y_prob_best = best_model.predict_proba(X_test)[:, 1]

print("Best Random Forest Model:")
print("Best Parameters:", best_params)
print("Classification Report:\n", classification_report(y_test, y_pred_best))
precision_best, recall_best, _ = precision_recall_curve(y_test, y_prob_best)
pr_auc_best = auc(recall_best, precision_best)
print("Area Under Precision-Recall Curve (AUPRC):", pr_auc_best)
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_best))


Best Random Forest Model:
Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93        93
           1       0.98      0.89      0.93       104

    accuracy                           0.93       197
   macro avg       0.94      0.94      0.93       197
weighted avg       0.94      0.93      0.93       197

Area Under Precision-Recall Curve (AUPRC): 0.9775463786319708
ROC AUC Score: 0.964485111662531


# Credit Card Fraud Detection Project

## Introduction
The objective of this project was to build a machine learning model to identify fraudulent credit card transactions. The dataset contains transactions made by credit cards in September 2013 by European cardholders. The dataset is highly unbalanced, with the positive class (fraud) accounting for only 0.172% of all transactions.

## Key Highlights

### Data Loading
- Successfully loaded the credit card fraud dataset.

### Exploratory Data Analysis (EDA)
- Visualized the distribution of features using pair plots.
- Analyzed correlations between features using a correlation matrix.

### Data Preprocessing
- Checked for and handled any missing values.
- Normalized the `Amount` and `Time` features to ensure consistent scaling.
- Addressed class imbalance issues using undersampling and oversampling techniques (SMOTE).

### Model Building and Training
- Split the data into training and testing sets.
- Built and trained Logistic Regression and Random Forest models to classify transactions as fraudulent or genuine.

### Model Evaluation
- Evaluated the models' performance using metrics such as precision, recall, F1-score, and Area Under the Precision-Recall Curve (AUPRC).
- The Logistic Regression model achieved an AUPRC of `insert_logistic_regression_auprc`.
- The Random Forest model achieved an AUPRC of `insert_random_forest_auprc`.
- Applied hyperparameter tuning using Grid Search to optimize the Random Forest model.

### Visualization
- Created visualizations such as pair plots, confusion matrix, and Precision-Recall curves to assess model performance and feature relationships.

## Insights
- The Logistic Regression and Random Forest models successfully classified fraudulent transactions with a reasonable level of accuracy.
- Normalizing features and addressing class imbalance significantly improved model performance.
- Hyperparameter tuning further enhanced the model's effectiveness.

## Future Work
- **Experimentation**: Explore other classification algorithms like XGBoost, K-Nearest Neighbors (KNN), Decision Trees, or Support Vector Machines (SVM) to see if they provide better accuracy.
- **Hyperparameter Tuning**: Continue performing hyperparameter tuning to optimize model performance.
- **Ensemble Methods**: Combine predictions from multiple models (stacking, boosting, or bagging) to improve overall performance.
- **Deployment**: Consider deploying the model for real-time fraud detection using a web application or API.
- **Monitoring**: Continuously monitor the model's performance in production and update it periodically to adapt to evolving fraud patterns.

This project demonstrates the power of machine learning in solving classification problems using real-world datasets like the credit card fraud dataset. The insights gained from this project can be further leveraged to explore more advanced techniques and applications in the field of fraud detection.

---

Feel free to customize the markdown as needed. If you have any questions or need further assistance, I'm here to help! 😊
